In [1]:
from tensorflow import keras
import tensorflow as tf

2023-11-28 01:35:07.223809: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-28 01:35:07.266054: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-28 01:35:07.266089: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-28 01:35:07.267021: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-28 01:35:07.272769: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-28 01:35:07.273166: I tensorflow/core/platform/cpu_feature_guard.cc:1

In [2]:
model = keras.models.load_model('bees-wasps.h5')

In [3]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)

tflite_model = converter.convert()

with open('bees-wasps.tflite', 'wb') as f_out:
    f_out.write(tflite_model)

INFO:tensorflow:Assets written to: /tmp/tmpzvj8dd9b/assets


INFO:tensorflow:Assets written to: /tmp/tmpzvj8dd9b/assets
2023-11-28 01:35:10.922780: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:378] Ignored output_format.
2023-11-28 01:35:10.922811: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:381] Ignored drop_control_dependency.
2023-11-28 01:35:10.923465: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /tmp/tmpzvj8dd9b
2023-11-28 01:35:10.924550: I tensorflow/cc/saved_model/reader.cc:51] Reading meta graph with tags { serve }
2023-11-28 01:35:10.924563: I tensorflow/cc/saved_model/reader.cc:146] Reading SavedModel debug info (if present) from: /tmp/tmpzvj8dd9b
2023-11-28 01:35:10.926367: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:388] MLIR V1 optimization pass is not enabled
2023-11-28 01:35:10.927062: I tensorflow/cc/saved_model/loader.cc:233] Restoring SavedModel bundle.
2023-11-28 01:35:11.048036: I tensorflow/cc/saved_model/loader.cc:217] Running initializatio

In [4]:
!ls -lh

total 129M
-rw-rw-r-- 1 paul paul  86M Nov 27 22:15 bees-wasps.h5
-rw-rw-r-- 1 paul paul  43M Nov 28 01:35 bees-wasps.tflite
-rw-rw-r-- 1 paul paul 416K Nov 27 23:12 corto.jpeg
drwxrwxr-x 2 paul paul 4,0K Nov 27 23:11 data
-rw-rw-r-- 1 paul paul  264 Nov 28 00:33 Dockerfile
-rw-rw-r-- 1 paul paul  19K Nov 28 01:33 h9.ipynb
-rw-rw-r-- 1 paul paul 1,4K Nov 28 00:37 lambda_function.py


## Question 1
### The size is: 43Mb

In [5]:
import tensorflow.lite as tflite

In [6]:
interpreter = tflite.Interpreter(model_path='bees-wasps.tflite')
interpreter.allocate_tensors()

input_index = interpreter.get_input_details()[0]['index']
output_index = interpreter.get_output_details()[0]['index']

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


In [7]:
output_index

13

## Question 2
### The ouput index for this model is 13

In [2]:
from io import BytesIO
from urllib import request

from PIL import Image

def download_image(url):
    with request.urlopen(url) as resp:
        buffer = resp.read()
    stream = BytesIO(buffer)
    img = Image.open(stream)
    return img


def prepare_image(img, target_size):
    if img.mode != 'RGB':
        img = img.convert('RGB')
    img = img.resize(target_size, Image.NEAREST)
    return img

In [9]:
pip install pillow

Note: you may need to restart the kernel to use updated packages.


In [13]:
from keras_image_helper import create_preprocessor
from tensorflow.keras.applications.xception import preprocess_input
import numpy as np

In [14]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [15]:

img = download_image('https://habrastorage.org/webt/rt/d9/dh/rtd9dhsmhwrdezeldzoqgijdg8a.jpeg')
x = prepare_image(img, (150,150))
x = np.asarray(x)
print(x.shape)
x = x.reshape((1,) + x.shape)

datagen = ImageDataGenerator(rescale=1./255)
x = datagen.flow(x)
x[0]

(150, 150, 3)


array([[[[0.9450981 , 0.9058824 , 0.8588236 ],
         [0.93725497, 0.9215687 , 0.97647065],
         [0.91372555, 0.8980393 , 0.9568628 ],
         ...,
         [0.2901961 , 0.33333334, 0.16470589],
         [0.34901962, 0.40784317, 0.15294118],
         [0.29803923, 0.36078432, 0.11764707]],

        [[0.9490197 , 0.909804  , 0.8705883 ],
         [0.9176471 , 0.909804  , 0.9607844 ],
         [0.90196085, 0.8941177 , 0.9490197 ],
         ...,
         [0.27450982, 0.3372549 , 0.16078432],
         [0.47058827, 0.5058824 , 0.18823531],
         [0.45098042, 0.49411768, 0.18431373]],

        [[0.9294118 , 0.882353  , 0.8196079 ],
         [0.91372555, 0.909804  , 0.96470594],
         [0.9058824 , 0.8941177 , 0.9607844 ],
         ...,
         [0.32156864, 0.37647063, 0.1764706 ],
         [0.5058824 , 0.5254902 , 0.20392159],
         [0.43137258, 0.4666667 , 0.20392159]],

        ...,

        [[0.03137255, 0.06666667, 0.04705883],
         [0.02352941, 0.08627451, 0.04705883]

## Question 3
### Value in the first pixel, the R channel is 0.9450980

In [18]:
import numpy as np
import tensorflow.lite as tflite
from keras_image_helper import create_preprocessor

interpreter = tflite.Interpreter(model_path='bees-wasps.tflite')
interpreter.allocate_tensors()

input_index = interpreter.get_input_details()[0]['index']
output_index = interpreter.get_output_details()[0]['index']

interpreter.set_tensor(input_index, x[0])
interpreter.invoke()
preds = interpreter.get_tensor(output_index)


In [24]:
x[0].shape

(1, 150, 150, 3)

In [19]:
preds

array([[0.6589842]], dtype=float32)

## Question 4
### The output of the model is 0.65898

## Question 5
### The size of the base image from agrigorev/zoomcamp-bees-wasps:v2 is 662Mb

In [11]:
import numpy as np
import tflite_runtime.interpreter as tflite
from keras_image_helper import create_preprocessor
from tensorflow.keras.preprocessing.image import ImageDataGenerator

def load_img(url, target_size=(150,150), rescale=1./255):
    img = download_image(url)
    x = prepare_image(img, (150,150))
    x = np.asarray(x,dtype=np.float32)
    x = x.reshape((1,) + x.shape)
    x = x*rescale
    return x

interpreter = tflite.Interpreter(model_path='bees-wasps.tflite')
input_index = interpreter.get_input_details()[0]['index']
output_index = interpreter.get_output_details()[0]['index']
interpreter.allocate_tensors()

def predict(url):
    x=load_img(url)
    interpreter.set_tensor(input_index, x)
    interpreter.invoke()
    preds = interpreter.get_tensor(output_index)
    float_predictions = preds[0].tolist()
    return float_predictions

url='https://habrastorage.org/webt/rt/d9/dh/rtd9dhsmhwrdezeldzoqgijdg8a.jpeg'
predict(url)

[0.6589841842651367]

## Question 6
### Using the alternative model we get an output of 0.4453